In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

C:\Users\User\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
dfnull = pd.read_csv('TOP_FEATURES2.csv')
dfnull.head()

,LSA1,LSA2,LSA3,LSA4,LSA5,LSA6,LSA7,LSA8,LSA9,LSA10,...,FASTTEXT248,FASTTEXT249,FASTTEXT250,FASTTEXT251,FASTTEXT252,FASTTEXT253,FASTTEXT254,FASTTEXT255,FASTTEXT256,TARGET
0,0.396314,-0.036150,0.064518,-0.073159,0.096052,-0.050067,-0.090869,0.037282,0.100209,0.046334,...,-0.000002,0.000001,0.000263,0.000348,-0.000052,-0.000270,-0.000334,0.000353,0.000310,0
1,0.401282,-0.186660,-0.115922,-0.267420,-0.016475,-0.064113,-0.072102,-0.035793,0.172962,-0.108011,...,-0.000108,0.000068,0.000095,-0.000318,0.000315,0.000111,-0.000155,0.000010,0.000061,0
2,0.421012,-0.051170,0.031429,-0.184141,-0.075076,-0.184510,-0.078020,0.011355,-0.018534,0.067499,...,0.000265,0.000006,0.000012,-0.000199,0.000148,0.000175,-0.000399,0.000139,-0.000191,0
3,0.419083,0.137935,0.029789,0.027398,0.053296,0.017428,-0.058439,-0.094601,0.061460,-0.050057,...,-0.000055,0.000263,-0.000010,-0.000090,-0.000160,-0.000041,-0.000250,0.000085,-0.000165,0
4,0.411648,0.156481,0.054695,-0.188321,0.130529,-0.111796,0.059512,-0.038671,0.045371,-0.163627,...,-0.000156,0.000129,-0.000020,0.000163,0.000375,0.000178,-0.000314,-0.000084,-0.000133,0


In [3]:
dfnull.shape

(1426, 929)

In [4]:
df = dfnull.dropna()

In [5]:
df.isnull().sum()

LSA1           0
LSA2           0
LSA3           0
LSA4           0
LSA5           0
              ..
FASTTEXT253    0
FASTTEXT254    0
FASTTEXT255    0
FASTTEXT256    0
TARGET         0
Length: 929, dtype: int64

In [6]:
df["TARGET"].head()

0    0
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

In [7]:
y = df["TARGET"]
x = df.drop("TARGET", axis=1) 
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [8]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.827074  0.674285   
1  XGBClassifier(base_score=None, booster=None, c...  0.876369  0.753270   
2      LGBMClassifier(max_depth=10, random_state=50)  0.885759  0.775667   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.852895  0.706136   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.686228  0.373469   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.654147   0.931818  0.705790  0.803206     0.948357     0.705790  
1  0.752739   0.891057  0.857590  0.874003     0.895149     0.857590  
2  0.771518   0.930192  0.834116  0.879538     0.937402     0.834116  
3  0.705790   0.864297  0.837246  0.850556     0.868545     0.837246  
4  0.372457   0.673469  0.723005  0.697358     0.649452     0.723005  


PCA_100

In [11]:
df.shape

(1424, 929)

In [25]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)

x_pca = pca.fit_transform(x)
x_pca.shape

(1424, 100)

In [30]:
xtrain_pca_100, xtest_pca_100 , ytrain, ytest = train_test_split(x_pca , y, test_size=0.2)

In [31]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain_pca_100, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.833333  0.675411   
1  XGBClassifier(base_score=None, booster=None, c...  0.843096  0.687011   
2      LGBMClassifier(max_depth=10, random_state=50)  0.860530  0.722185   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.806137  0.612466   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.687587  0.375421   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.666667   0.897010  0.753138  0.818802     0.913529     0.753138  
1  0.686192   0.860704  0.818689  0.839171     0.867503     0.818689  
2  0.721060   0.881832  0.832636  0.856528     0.888424     0.832636  
3  0.612273   0.798639  0.818689  0.808540     0.793584     0.818689  
4  0.375174   0.694645  0.669456  0.681818     0.705718     0.669456  


PCA_50

In [32]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)

x_pca = pca.fit_transform(x)
x_pca.shape

xtrain_pca_50, xtest_pca_50 , ytrain, ytest = train_test_split(x_pca , y, test_size=0.2)

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain_pca_50, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.792732  0.586153   
1  XGBClassifier(base_score=None, booster=None, c...  0.786676  0.573352   
2      LGBMClassifier(max_depth=10, random_state=50)  0.794751  0.589528   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.769852  0.541593   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.690444  0.382054   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.585464   0.807638  0.768506  0.787586     0.816958     0.768506  
1  0.573351   0.786290  0.787349  0.786819     0.786003     0.787349  
2  0.589502   0.792000  0.799462  0.795713     0.790040     0.799462  
3  0.539704   0.749068  0.811575  0.779070     0.728129     0.811575  
4  0.380888   0.676654  0.729475  0.702073     0.651413     0.729475  


PCA_200

In [33]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)

x_pca = pca.fit_transform(x)
x_pca.shape

xtrain_pca_200, xtest_pca_200 , ytrain, ytest = train_test_split(x_pca , y, test_size=0.2)

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain_pca_200, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.781507  0.563133   
1  XGBClassifier(base_score=None, booster=None, c...  0.770548  0.541109   
2      LGBMClassifier(max_depth=10, random_state=50)  0.766438  0.533021   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.749315  0.501073   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.632192  0.267055   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.563014   0.787413  0.771233  0.779239     0.791781     0.771233  
1  0.541096   0.768707  0.773973  0.771331     0.767123     0.773973  
2  0.532877   0.760375  0.778082  0.769127     0.754795     0.778082  
3  0.498630   0.726933  0.798630  0.761097     0.700000     0.798630  
4  0.264384   0.615846  0.702740  0.656430     0.561644     0.702740  


PCA_97%

In [35]:
from sklearn.decomposition import PCA
pca = PCA(0.97)

x_pca = pca.fit_transform(x)
x_pca.shape

xtrain_pca_97p, xtest_pca_97p , ytrain, ytest = train_test_split(x_pca , y, test_size=0.2)

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain_pca_97p, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.841128  0.692235   
1  XGBClassifier(base_score=None, booster=None, c...  0.859010  0.720236   
2      LGBMClassifier(max_depth=10, random_state=50)  0.870702  0.742418   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.823934  0.648283   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.678129  0.356260   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.682256   0.910596  0.756534  0.826446     0.925722     0.756534  
1  0.718019   0.889552  0.819807  0.853257     0.898212     0.819807  
2  0.741403   0.891147  0.844567  0.867232     0.896836     0.844567  
3  0.647868   0.812749  0.841816  0.827027     0.806052     0.841816  
4  0.356259   0.678621  0.676754  0.677686     0.679505     0.676754  


In [36]:
xtrain_pca_97p.shape

(1139, 104)

PCA_98%

In [42]:
from sklearn.decomposition import PCA
pca = PCA(0.98)

x_pca = pca.fit_transform(x)
x_pca.shape

xtrain_pca_98p, xtest_pca_98p , ytrain, ytest = train_test_split(x_pca , y, test_size=0.2)

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain_pca_98p, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.817227  0.652105   
1  XGBClassifier(base_score=None, booster=None, c...  0.852241  0.707991   
2      LGBMClassifier(max_depth=10, random_state=50)  0.860644  0.726212   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.841036  0.682368   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.703081  0.406475   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.634454   0.912568  0.701681  0.793349     0.932773     0.701681  
1  0.704482   0.891135  0.802521  0.844510     0.901961     0.802521  
2  0.721289   0.908082  0.802521  0.852045     0.918768     0.802521  
3  0.682073   0.851371  0.826331  0.838664     0.855742     0.826331  
4  0.406162   0.711370  0.683473  0.697143     0.722689     0.683473  


In [43]:
xtrain_pca_98p.shape

(1139, 142)

PCA_99%

In [48]:
from sklearn.decomposition import PCA
pca = PCA(0.99)

x_pca = pca.fit_transform(x)
x_pca.shape

xtrain_pca_99p, xtest_pca_99p , ytrain, ytest = train_test_split(x_pca , y, test_size=0.2)

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain_pca_99p, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.824061  0.668701   
1  XGBClassifier(base_score=None, booster=None, c...  0.840056  0.683358   
2      LGBMClassifier(max_depth=10, random_state=50)  0.853964  0.713795   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.825452  0.651086   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.678720  0.357509   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.648122   0.929889  0.700974  0.799366     0.947149     0.700974  
1  0.680111   0.876733  0.791377  0.831871     0.888734     0.791377  
2  0.707928   0.905901  0.789986  0.843982     0.917942     0.789986  
3  0.650904   0.833333  0.813630  0.823364     0.837274     0.813630  
4  0.357441   0.675307  0.688456  0.681818     0.668985     0.688456  


In [49]:
xtrain_pca_99p.shape

(1139, 208)